In [1]:
 import warnings
warnings.filterwarnings('ignore')

In [2]:
 import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
 from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
 # https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [83]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [96]:
#Dummy variables
X = 


#Removinngg all the 'str' data that was incompatable with Scaler. spent an hour trying to convert the 85 columns str to float but gave up. Selecting all 'int' columns insteaad - BJORN
df['home_ownership'] = pd.to_numeric(df['home_ownership'],errors='coerce')
df['verification_status'] = pd.to_numeric(df['verification_status'],errors='coerce')
df = df.filter(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'loan_status', 'dti', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit'])
print(df)

       loan_amnt  int_rate  installment  annual_inc loan_status    dti  \
0        10500.0    0.1719       375.35     66000.0    low_risk  27.24   
1        25000.0    0.2000       929.09    105000.0    low_risk  20.23   
2        20000.0    0.2000       529.88     56000.0    low_risk  24.26   
3        10000.0    0.1640       353.55     92000.0    low_risk  31.44   
4        22000.0    0.1474       520.39     52000.0    low_risk  18.76   
...          ...       ...          ...         ...         ...    ...   
68812    10000.0    0.1502       346.76     26000.0    low_risk   9.60   
68813    12000.0    0.2727       368.37     63000.0    low_risk  29.07   
68814     5000.0    0.1992       185.62     52000.0    low_risk  14.86   
68815    40000.0    0.0646      1225.24    520000.0    low_risk   9.96   
68816    16000.0    0.1131       350.36     72000.0    low_risk   7.02   

       pct_tl_nvr_dlq  percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  \
0                85.7             1

TypeError: 'Series' object is not callable

In [91]:
 # Months dictionary
#import calendar
#name_to_num = {name: num for num, name in enumerate(calendar.month_name) if num}
#name_to_num
 # Encode month name
#loans_df["Month"] = loans_df["Month"].apply(lambda x: name_to_num[x])
#loans_df.head()

NameError: name 'loans_df' is not defined

In [97]:
# Create our features

y = df["loan_status"]
X = df.drop(columns="loan_status")


# Split the Data into Training and Testing # Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape



(51612, 13)

In [98]:
 # Check the balance of our target values
df['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [99]:
 # Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [101]:
 # Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
scaler.fit_transform(X_train)

array([[-0.64978133, -0.30327574, -0.52669276, ..., -0.09170382,
        -0.75733047, -0.03294137],
       [ 0.42378352, -0.42616892,  0.71269346, ..., -0.44760726,
         1.3046767 , -0.50474625],
       [ 1.79013878, -0.82400957,  2.19335392, ..., -1.04470431,
         0.438036  , -0.57928896],
       ...,
       [ 0.81417074, -0.36368086,  1.17925683, ..., -0.36245362,
         0.40441632, -0.44272156],
       [-0.25939411, -0.42616892, -0.08220905, ..., -0.20693308,
        -0.70129766,  0.15147531],
       [-0.16179731, -1.196855  , -0.05849257, ..., -0.40364526,
         0.1242523 , -0.46649022]])

In [103]:
 # Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [108]:
 # Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
 # Create the random forest classifier instance
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
# Fit the model
rf_model = rf_model.fit(X_train_scaled, y_train)


In [109]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [112]:
 # Calculating the confusion matrix
    # YOUR CODE HERE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report    
    
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)


In [117]:
# # Print the classification report
# YOUR CODE HERE
print("Confusion Matrix = ")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report = ")
print(classification_report(y_test, predictions))

# Print the imbalanced classification report
# YOUR CODE HERE
#y_pred_rf = brf.predict(X_test)
print("Classification Report IMBALANCED = ")
print(classification_report_imbalanced(y_test, predictions))

Confusion Matrix = 


,Predicted 0,Predicted 1
Actual 0,0,87
Actual 1,0,17118


Accuracy Score : 0.9949433304272014
Classification Report = 
              precision    recall  f1-score   support

   high_risk       0.00      0.00      0.00        87
    low_risk       0.99      1.00      1.00     17118

    accuracy                           0.99     17205
   macro avg       0.50      0.50      0.50     17205
weighted avg       0.99      0.99      0.99     17205

Classification Report IMBALANCED = 
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.00      0.00      1.00      0.00      0.00      0.00        87
   low_risk       0.99      1.00      0.00      1.00      0.00      0.00     17118

avg / total       0.99      0.99      0.01      0.99      0.00      0.00     17205



In [114]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
# Get the feature importance array
importances = rf_model.feature_importances_

# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.12852676013190759, 'tot_hi_cred_lim'),
 (0.12382196164533424, 'total_bal_ex_mort'),
 (0.12246407938412737, 'dti'),
 (0.1156766841858365, 'total_il_high_credit_limit'),
 (0.10091897898717782, 'total_bc_limit'),
 (0.09950662386898085, 'installment'),
 (0.08870007693110407, 'annual_inc'),
 (0.0647856282926732, 'loan_amnt'),
 (0.05449437658399925, 'int_rate'),
 (0.052266093638417575, 'pct_tl_nvr_dlq')]

In [118]:
# Train the EasyEnsembleClassifier -- We're using a "BalancedRandomForestClassifier" as there was no "EasyEnsembleClassifier" example on Git -- similar concept though. -- BJORN
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)


BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [ ]:
 # Calculated the balanced accuracy score
# YOUR CODE HERE
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [120]:
 # Display the confusion matrix
# YOUR CODE HERE
print("Confusion Matrix = ")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report = ")
print(classification_report(y_test, predictions))

Confusion Matrix = 


,Predicted 0,Predicted 1
Actual 0,0,87
Actual 1,0,17118


Accuracy Score : 0.9949433304272014
Classification Report = 
              precision    recall  f1-score   support

   high_risk       0.00      0.00      0.00        87
    low_risk       0.99      1.00      1.00     17118

    accuracy                           0.99     17205
   macro avg       0.50      0.50      0.50     17205
weighted avg       0.99      0.99      0.99     17205



In [119]:
# Print the imbalanced classification report
# YOUR CODE HERE
y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.55      0.57      0.01      0.56      0.32        87
   low_risk       1.00      0.57      0.55      0.73      0.56      0.32     17118

avg / total       0.99      0.57      0.55      0.72      0.56      0.32     17205

